In [ ]:
import os
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from train.data import DeepCleanDataset
from train.model import DeepClean
import torch

from train.architectures import Architecture
from train.metrics import OnlinePsdRatio, PsdRatio
from collections.abc import Callable
from typing import Optional

from ml4gw.transforms import SpectralDensity
from torchmetrics import Metric


In [ ]:
def train_deepclean(config, data_dir=None, num_epochs=10, num_gpus=0):
    model = DeepClean(
        arch=Architecture(),
        loss=PsdRatio(
            sample_rate=config["sample_rate"],
            fftlength=config["fftlength"],
            freq_low=config["freq_low"],
            freq_high=config["freq_high"],
            asd=config["asd"]
        ),
        metric=OnlinePsdRatio(
            inference_sampling_rate=config["inference_sampling_rate"],
            edge_pad=config["edge_pad"],
            filter_pad=config["filter_pad"],
            sample_rate=config["sample_rate"],
            bandpass=Callable,  # Define or import your bandpass function
            y_scaler=torch.nn.Module  # Define or import your scaler function
        ),
        patience=config["patience"],
        save_top_k_models=config["save_top_k_models"]
    )

    data_module = DeepCleanDataset(
        fname=os.path.join(data_dir, 'data/K-K1_lldata-1369291863-12288.hdf5'),
        channels=['K1:CAL-CS_PROC_DARM_STRAIN_DBL_DQ', 'K1:PEM-MIC_OMC_BOOTH_OMC_Z_OUT_DQ'],
        kernel_length=config["kernel_length"],
        freq_low=[config["freq_low"]],
        freq_high=[config["freq_high"]],
        batch_size=int(config["batch_size"]),
        train_duration=4096,
        test_duration=8192,
        valid_frac=0.33,
        train_stride=0.0625,
        inference_sampling_rate=64,
        start_offset=0,
        filt_order=8
    )

    trainer = Trainer(
        logger=TensorBoardLogger(save_dir=tune.get_trial_dir(), name="", version="."),
        max_epochs=num_epochs,
        gpus=num_gpus,
        callbacks=[
            TuneReportCallback({"loss": "val_loss"}, on="validation_end")
        ]
    )

    trainer.fit(model, datamodule=data_module)

def tune_hyperparameters(num_samples=10, num_epochs=10, gpus_per_trial=0):
    config = {
        "sample_rate": tune.choice([2048, 4096]),
        "fftlength": tune.choice([2, 4]),
        "freq_low": tune.choice([55, 58]),
        "freq_high": tune.choice([62, 65]),
        # "asd": tune.choice([True, False]),
        "inference_sampling_rate": tune.choice([64, 4]),
        # "edge_pad": tune.uniform(0.1, 0.5),
        # "filter_pad": tune.uniform(0.1, 0.5),
        "patience": tune.choice([10, 20]),
        "save_top_k_models": tune.choice([1, 3]),
        "batch_size": tune.choice([32, 512]),
        "max_epochs": tune.choice([20, 100])
    }

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=num_epochs,
        grace_period=1,
        reduction_factor=2)

    analysis = tune.run(
        tune.with_parameters(
            train_deepclean,
            data_dir="/home/shuwei.yeh/deepclean/data",
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial),
        resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
        metric="loss",
        mode="min",
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=tune.CLIReporter(parameter_columns=list(config.keys()))
    )

    print("Best hyperparameters found were: ", analysis.best_config)

if __name__ == "__main__":
    # Set the number of samples and epochs for tuning
    tune_hyperparameters(num_samples=10, num_epochs=10, gpus_per_trial=1)